In [1]:
import calendar
import json
from collections import Counter
from datetime import date
from pprint import pprint
import os
from pymongo import MongoClient

In [ ]:
with open(os.path.expanduser('~/.mongo/mongo.json'),'r') as config:
    mongo_uri = json.load(config)['mongo_uri']
client = MongoClient(mongo_uri)
db = client.forethought

organization_id = '440'
start = '2021-06-09T00:00:00Z'
end = '2021-06-029T00:00:00Z'
mongo_org_collection = db[str(organization_id)]
escalation = ["roadie_escalations"]

In [ ]:
def get_dataset():
    tickets = mongo_org_collection.find({
        'doc_type': 'ticket',
        'is_deleted': False,
        'original_item.via.channel': {'$in':['web','email', 'api']},
        'original_item.created_at': {'$gte': start, '$lte': end},
        "original_item.tags": {"$in": escalation}
    })
    
    file_path = os.path.expanduser('~/Downloads/rdie_esc.json')
    with open(file_path, 'w') as f:
        count = 0
        for ticket in tickets:
            i = ticket["original_item"]
            variable_labels = [x for x in escalation if x in i['tags']]
            j = {
                "text_a": i["subject"],
                "text_b": i["description"],
                "created_at": i["created_at"],
                "label": variable_labels[0] if len(variable_labels) > 0 else None,
                "labels": sorted(variable_labels)
                }
                
            json.dump(j, f)
            _ = f.write('\n')
            count += 1
            if count % 100 == 0:
                print(count)

    print("total: ", count)


get_dataset()